In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
from torch import nn, optim

In [3]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [4]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

def make_acc_dataset(model, dataset):
  n = len(dataset)
  for i in range(n):
      x, y = dataset[i][0], dataset[i][1]
      with torch.no_grad():
          pred = model(x)
      if (not(pred[0].argmax(0) - y)):
          dataset.targets[i] = 1
      else:
          dataset.targets[i] = 0
  return dataset

In [5]:
# As we save
device = "cpu"
print("Using {} device".format(device))
# Define model
class NeuralNetworkClass(nn.Module):
    def __init__(self):
        super(NeuralNetworkClass, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x).to(device)
        return logits

class_model = NeuralNetworkClass().to(device)
print(class_model)

class NeuralNetworkPred(nn.Module):
    def __init__(self):
        super(NeuralNetworkPred, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 2),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x).to(device)
        return logits
pred_model = NeuralNetworkPred().to(device)
print(pred_model)

Using cpu device
NeuralNetworkClass(
  (flatten): Flatten()
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)
NeuralNetworkPred(
  (flatten): Flatten()
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=2, bias=True)
    (5): ReLU()
  )
)


In [20]:
path = '/home/amnesiac/PycharmProjects/jamesnick/notebooks'
model = NeuralNetworkPred()
model.load_state_dict(torch.load(path + '/1' + '_pred_model.pth'))

RuntimeError: Error(s) in loading state_dict for NeuralNetworkPred:
	size mismatch for linear_relu_stack.0.weight: copying a param with shape torch.Size([10, 784]) from checkpoint, the shape in current model is torch.Size([512, 784]).
	size mismatch for linear_relu_stack.0.bias: copying a param with shape torch.Size([10]) from checkpoint, the shape in current model is torch.Size([512]).
	size mismatch for linear_relu_stack.2.weight: copying a param with shape torch.Size([10, 10]) from checkpoint, the shape in current model is torch.Size([512, 512]).
	size mismatch for linear_relu_stack.2.bias: copying a param with shape torch.Size([10]) from checkpoint, the shape in current model is torch.Size([512]).
	size mismatch for linear_relu_stack.4.weight: copying a param with shape torch.Size([2, 10]) from checkpoint, the shape in current model is torch.Size([2, 512]).

In [14]:
# Lets make a function that loads the models into a python list
def load_acc_models(path,n_of_models,model):
    model_list = []
    for i in range(n_of_models):
        new_model = model()
        new_model.load_state_dict(torch.load(path + i + '_pred_model.pth'))
        model_list.append(new_model)
    return model_list
        
        
    

In [19]:
# 
path = '/home/amnesiac/PycharmProjects/jamesnick/notebooks'
n_of_models = 10
model = NeuralNetworkPred()

list_of_acc_model = load_acc_models(path,n_of_models,model)

TypeError: forward() missing 1 required positional argument: 'x'